In [1]:
from pynput import keyboard
import sounddevice as sd
import numpy as np
import tensorflow as tf
import librosa
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

2025-03-04 20:36:47.599067: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741109807.614404   67471 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741109807.617825   67471 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-04 20:36:47.631569: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
SAMPLE_RATE = 44100  # Частота дискретизации
commands = ['eight', 'stop', 'zero', 'up', 'right', 'three', 'on', 'two', 'nine', 'seven', 'four', 'six', 'yes', 'go', 'five', 'one', 'left', 'down', 'no', 'off']
recording = False
buffer = []

In [3]:
# Функция обратного вызова для записи звука
def callback(indata, frames, time, status):
    if recording:
        buffer.extend(indata[:, 0])
        
# Обработчик нажатия клавиши ALT
def on_press_alt(key):
    global recording, buffer
    if key == keyboard.Key.alt:
        if not recording:
            print("Началась запись. Говорите команду...")
            buffer = []
            recording = True

# Обработчик отпускания клавиши ALT
def on_release_alt(key):
    global recording
    if key == keyboard.Key.alt:
        if recording:
            print("Запись завершена.")
            recording = False
            process_audio()

In [4]:
def process_audio():
    global buffer # Используем глобальный буфер для хранения записанных данных
    audio = np.array(buffer, dtype=np.float32)  # Преобразуем буфер в массив numpy с типом float32

    # Проверяем, есть ли в аудио речь
    if is_silent(audio, 0.03):
        print("Нет речи, повторите команду...")
        return

    # Изменяем частоту дискретизации аудиофайла с 44.1 кГц до 16 кГц
    audio_resampled = librosa.resample(audio, orig_sr=44100, target_sr=16000)

    # Передаем обработанное аудио в модель для распознавания команды
    command_id, confidence= predict_command(loaded_model, audio_resampled)
    
    print(f"Распознано: {commands[command_id]} с уверенностью {float(confidence):.3f}")

In [5]:
def get_mel_spectrogram(waveform):
    # Обрезаем или заполняем нулями до 16000 сэмплов
    input_len = 16000
    waveform = waveform[:input_len]
    zero_padding = tf.zeros([16000] - tf.shape(waveform), dtype=tf.float32)
    waveform = tf.concat([waveform, zero_padding], 0)

    # Применяем STFT
    spectrogram = tf.signal.stft(waveform, frame_length=400, frame_step=160)
    spectrogram = tf.abs(spectrogram)

    # Создаем фильтр мел-шкал (80 мел-фильтров)
    num_mel_bins = 80
    linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(
        num_mel_bins=num_mel_bins,
        num_spectrogram_bins=spectrogram.shape[-1],
        sample_rate=16000,
        lower_edge_hertz=80.0,
        upper_edge_hertz=7600.0
    )

    # Применяем мел-фильтры для получения мел-спектрограммы
    mel_spectrogram = tf.tensordot(spectrogram, linear_to_mel_weight_matrix, 1)
    mel_spectrogram = tf.math.log(mel_spectrogram + 1e-6)  # Логарифм для стабильности

    mel_spectrogram = mel_spectrogram[..., tf.newaxis]  # Добавляем новое измерение
    return mel_spectrogram


In [6]:
def predict_command(model, waveform):
    spectrogram = get_mel_spectrogram(waveform)

    # Изменяем размер спектрограммы до (32, 32) с помощью tf.image.resize
    # Это необходимо, чтобы привести данные к размеру, ожидаемому моделью
    spectrogram = tf.image.resize(spectrogram, (32, 32))

    # Добавляем дополнительное измерение, чтобы спектрограмма имела форму (1, 32, 32, 1)
    # Это требуется, так как модель ожидает входные данные в формате (батч, высота, ширина, каналы)
    spectrogram = tf.expand_dims(spectrogram, axis=0)

    predictions = model(spectrogram, training=False) # Передаем спектрограмму в модель для получения предсказаний
    
    confidence = np.max(tf.nn.softmax(predictions).numpy()) # Получаем уверенность
    
    predicted_label = tf.argmax(predictions, axis=1).numpy()[0] # Находим индекс класса с наибольшей вероятностью
    
    return predicted_label, confidence

In [7]:
def is_silent(audio, threshold=0.02):
    # Вычисляем среднеквадратичное значение (RMS) аудиосигнала
    # RMS — это мера средней мощности сигнала
    rms = np.sqrt(np.mean(audio**2))
    return rms < threshold  # Если RMS меньше заданного порога, считаем сигнал тихим

In [8]:
device_info = sd.query_devices(sd.default.device, 'input')
print("Доступные частоты дискретизации:", device_info['default_samplerate'])

Доступные частоты дискретизации: 44100.0


In [9]:
def resample_audio(audio, orig_sr=device_info['default_samplerate'], target_sr=16000):
    return librosa.resample(audio, orig_sr=orig_sr, target_sr=target_sr)

In [10]:
loaded_model = tf.keras.models.load_model('../Models/mel_cnn.keras')

2025-03-04 20:36:49.283287: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-03-04 20:36:49.283310: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:137] retrieving CUDA diagnostic information for host: flayven-MS-7C91
2025-03-04 20:36:49.283317: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:144] hostname: flayven-MS-7C91
2025-03-04 20:36:49.283406: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:168] libcuda reported version is: 550.120.0
2025-03-04 20:36:49.283422: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:172] kernel reported version is: 550.120.0
2025-03-04 20:36:49.283426: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:259] kernel version seems to match DSO: 550.120.0


In [11]:
try:
    with sd.InputStream(samplerate=SAMPLE_RATE, channels=1, dtype=np.float32, callback=callback):
        with keyboard.Listener(on_press=on_press_alt, on_release=on_release_alt) as listener:
            listener.join()
except KeyboardInterrupt:
    print("Программа завершена пользователем.")
except Exception as e:
    print(f"Произошла ошибка: {e}")

Началась запись. Говорите команду...
Запись завершена.
Распознано: up с уверенностью 0.999
Началась запись. Говорите команду...
Запись завершена.
Распознано: down с уверенностью 0.549
Началась запись. Говорите команду...
Запись завершена.
Распознано: left с уверенностью 0.863
Началась запись. Говорите команду...
Запись завершена.
Распознано: right с уверенностью 0.755
Началась запись. Говорите команду...
Запись завершена.
Распознано: yes с уверенностью 0.999
Началась запись. Говорите команду...
Запись завершена.
Распознано: zero с уверенностью 0.998
Началась запись. Говорите команду...
Запись завершена.
Распознано: one с уверенностью 0.929
Началась запись. Говорите команду...
Запись завершена.
Распознано: two с уверенностью 0.613
Началась запись. Говорите команду...
Запись завершена.
Распознано: three с уверенностью 0.946
Началась запись. Говорите команду...
Запись завершена.
Распознано: stop с уверенностью 0.473
Началась запись. Говорите команду...
Запись завершена.
Распознано: four с